# Data Analysis to understand which kind of  apps attract maximum users in appstores.



In [742]:
from csv import reader

apple_store= open('AppleStore.csv')
google_store= open('googleplaystore.csv')

read_apple_store = reader(apple_store)
read_google_store = reader(google_store)

apple_store_list = list(read_apple_store)
google_store_list = list(read_google_store)



# The explore_data() Function 

The following explore data function takes in a list with a start and end index prints the rows with a breakline in between. This makes it easier to see and analyse data visually. it also prints the number of rows and columns if the argument is given as TRUE for the fourth parameter.

In [743]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line between rows
        
    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [744]:
explore_data(apple_store_list,0,2,True)
explore_data(google_store_list,0,2,True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


Number of rows: 7198
Number of columns: 16
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


Number of rows: 10842
Number of columns: 13


# Deleting erroneous data
Dataset may sometimes contain erroneous data.The following code deletes a wrong data which was observed by the discussion page of google app store on kaggle.com

In [745]:
print(google_store_list[10473])

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


In [746]:
del(google_store_list[10473])


In [747]:
print(len(google_store_list))

10841


# Removing Duplicate entries

Every data set has a possibility of having duplicate entries.The following code identifies duplicate entries

In [748]:
google_duplicate_appnames=[]
google_unique_appnames=[]
for each in google_store_list[1:]:
    if each[0] in google_unique_appnames:
        google_duplicate_appnames.append(each[0])
    else:
        google_unique_appnames.append(each[0])
        
print('Number of Duplicate apps in google playstore : ',len(google_duplicate_appnames))
print('Number of Unique apps in google playstore : ',len(google_unique_appnames))

Number of Duplicate apps in google playstore :  1181
Number of Unique apps in google playstore :  9659


In [749]:
apple_duplicate_appnames=[]
apple_unique_appnames=[]
for each in apple_store_list[1:]:
    if each[0] in apple_unique_appnames:
        apple_duplicate_appnames.append(each[0])
    else:
        apple_unique_appnames.append(each[0])
        
print('Number of Duplicate apps in apple store : ',len(apple_duplicate_appnames))
print('Number of Unique apps in apple store : ',len(apple_unique_appnames))

Number of Duplicate apps in apple store :  0
Number of Unique apps in apple store :  7197


###### In order to have a high level of accuracy in data analysis, we need to remove the duplicate values. I don't plan on removing duplicate values randomly.If you observe the columns of the data, you will find columns  reviews, installs, last updated etc. We need the most recent data which can be filtered on the basis of 'last updated' or by sorting the amount of reviews to get the record with max reviews. All other duplicated data will be then deleted


In [750]:
reviews_max={}

for each in google_store_list[1:]:
    name = each[0]
    n_reviews = float(each[3])
    if name in reviews_max and reviews_max[name]<n_reviews:
        reviews_max[name]=n_reviews
    elif(name not in reviews_max) :
        reviews_max[name]=n_reviews

 


###### Here we filter the data based on which apps have the maximum number of reviews so that it is the most recent data collected

In [751]:
android_clean=[]
already_added=[]

for each in google_store_list[1:]:
    name=each[0]
    n_reviews = float(each[3])
    if n_reviews == reviews_max[name] and name not in already_added:
        android_clean.append(each)
        already_added.append(name)
        
print(len(android_clean))

9659


# Removing Non English Apps

The folloing function takes a string as an input parameter and return false if there is any character in the string that doesn't belong to the set of common English characters. Otherwise it returns true.

In [752]:
def check_eng(str):
    for each in str:
        if(ord(each)>127):
            return False
    return True

print(check_eng('instagram'))
print(check_eng('爱奇艺PPS -《欢乐颂2》电视剧热播'))

True
False


The above function we wrote returns any app name as non english app even if it has just one character which is out of ascii range

For example, if the appname contains an emoji in the name which is just one character, but the app is english, it will still be returned as a non english app

A not so accurate but a fairly effective approach is to set a threshold count for out of range ascii character. Let's say if the app name has more than 3 out of range ascii character, only then should it return False. otherwise it should allow it to be in the category of English apps. This is not a very accurate approach but a bit better

In [753]:
def check_eng(str):
    count=0
    for each in str:
        if(ord(each)>127):
            count+=1
    if(count>3):
        return False
    else:
        return True


print(check_eng('Instachat 😜'))
print(check_eng('Docs To Go™ Free Office Suite'))

True
True


#### Now we use the above function to filter non-english apps from both google and apple data sets.

In [754]:
def check_eng(str):
    count=0
    for each in str:
        if(ord(each)>127):
            count+=1
    if(count>3):
        return False
    else:
        return True



english_android_list=[]
english_ios_list=[]


for each in android_clean:
    name = each[0]
    if(check_eng(name)):
        english_android_list.append(each)
        
#REMEMBER THE NAME OF THE APP IN APPLE IOS APPS LIST IS IN THE SECOND COLUMN
for each in apple_store_list[1:]:
    name = each[1]
    if(check_eng(name)):
        english_ios_list.append(each)
        
print('English android apps with no duplicate values : ',len(english_android_list))
print('English IOS apps with no duplicate values : ',len(english_ios_list))
#print(len(apple_store_list))
#print(len(android_clean))

        

English android apps with no duplicate values :  9614
English IOS apps with no duplicate values :  6183


In [755]:
free_android_apps=[]
free_ios_apps=[]

for each in english_android_list:
    
    if each[7]=='0':
        free_android_apps.append(each)
        
        
for each in english_ios_list:
    price = float(each[4])
    if price==0.0:
        free_ios_apps.append(each)
        
print('Number of free Android apps : ',len(free_android_apps))
print('Number of free IOS apps :', len(free_ios_apps))

Number of free Android apps :  8864
Number of free IOS apps : 3222


### Now we want to find an app profile that fits both the app store and google play

#### Our aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.

#### To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

#### Build a minimal Android version of the app, and add it to Google Play.

If the app has a good response from users, we develop it further.
If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.

In [756]:
print('Android app list columns:\n', google_store_list[0])
print('\n')
print('IOS app list columns:\n',apple_store_list[0])

Android app list columns:
 ['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


IOS app list columns:
 ['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


In [757]:
def freq_table(dataset,index):
    dict_freq = {}
    total=0
    for each in dataset:
        total+=1
        value = each[index]
        if value in dict_freq:
            dict_freq[value]+=1
        else:
            dict_freq[value]=1
    
    freq_table_percentages = {}
    for key in dict_freq:
        percentage = (dict_freq[key] / total) * 100
        freq_table_percentages[key] = percentage 
    
    return freq_table_percentages
    

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])



            

In [758]:
# Genre column for android apps
display_table(free_android_apps,9)


Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

In [759]:
#Category for andriod apps column
display_table(free_android_apps,1)



FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

In [760]:
#prime_genre column for IOS apps
display_table(free_ios_apps,11)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


#### In the code below, the variables are used as follows

len_genre = no. of apps belonging to that particular genre

n_ratings = total no. of ratings for that app

total= The total number of ratings to all the apps belonging to that genre

avg_n_ratings = Total / len_genre(Total number of ratings  belonging to that genre divided by no . of apps of that genre)


In [761]:
ios_genre_percentage = freq_table(free_ios_apps,11)

for genre in ios_genre_percentage:
    total=0
    len_genre = 0
    for app in free_ios_apps:
        app_genre = app[-5]
        if genre == app_genre:
            len_genre +=1
            n_ratings = float(app[5])
            total+=n_ratings
    avg_n_ratings = total/len_genre
    print(genre, ':', avg_n_ratings)

Navigation : 86090.33333333333
Games : 22788.6696905016
Food & Drink : 33333.92307692308
Finance : 31467.944444444445
Photo & Video : 28441.54375
Reference : 74942.11111111111
Education : 7003.983050847458
Sports : 23008.898550724636
Social Networking : 71548.34905660378
Medical : 612.0
Entertainment : 14029.830708661417
Health & Fitness : 23298.015384615384
Lifestyle : 16485.764705882353
Business : 7491.117647058823
News : 21248.023255813954
Book : 39758.5
Utilities : 18684.456790123455
Music : 57326.530303030304
Catalogs : 4004.0
Productivity : 21028.410714285714
Shopping : 26919.690476190477
Weather : 52279.892857142855
Travel : 28243.8


# Most Popular Apps by genre on App store. 

#### Below we recommend a profitable app profile for App Store

In [762]:
for each in free_ios_apps:
    if(each[-5]=='Reference'):
        print(each[1],each[5])
    

Bible 985920
Dictionary.com Dictionary & Thesaurus 200047
Dictionary.com Dictionary & Thesaurus for iPad 54175
Google Translate 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition 17588
Merriam-Webster Dictionary 16849
Night Sky 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools 4693
GUNS MODS for Minecraft PC Edition - Mods Tools 1497
Guides for Pokémon GO - Pokemon GO News and Cheats 826
WWDC 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free 718
VPN Express 14
Real Bike Traffic Rider Virtual Reality Glasses 8
教えて!goo 0
Jishokun-Japanese English Dictionary & Translator 0


In [763]:
for each in free_ios_apps:
    if(each[-5]=='Social Networking'):
        print(each[1],each[5])

Facebook 2974676
Pinterest 1061624
Skype for iPhone 373519
Messenger 351466
Tumblr 334293
WhatsApp Messenger 287589
Kik 260965
ooVoo – Free Video Call, Text and Voice 177501
TextNow - Unlimited Text + Calls 164963
Viber Messenger – Text & Call 164249
Followers - Social Analytics For Instagram 112778
MeetMe - Chat and Meet New People 97072
We Heart It - Fashion, wallpapers, quotes, tattoos 90414
InsTrack for Instagram - Analytics Plus More 85535
Tango - Free Video Call, Voice and Chat 75412
LinkedIn 71856
Match™ - #1 Dating App. 60659
Skype for iPad 60163
POF - Best Dating App for Conversations 52642
Timehop 49510
Find My Family, Friends & iPhone - Life360 Locator 43877
Whisper - Share, Express, Meet 39819
Hangouts 36404
LINE PLAY - Your Avatar World 34677
WeChat 34584
Badoo - Meet New People, Chat, Socialize. 34428
Followers + for Instagram - Follower Analytics 28633
GroupMe 28260
Marco Polo Video Walkie Talkie 27662
Miitomo 23965
SimSimi 23530
Grindr - Gay and same sex guys chat, meet

In [764]:
for each in free_ios_apps:
    if(each[-5]=='Navigation'):
        print(each[1],each[5])

Waze - GPS Navigation, Maps & Real-time Traffic 345046
Google Maps - Navigation & Transit 154911
Geocaching® 12811
CoPilot GPS – Car Navigation & Offline Maps 3582
ImmobilienScout24: Real Estate Search in Germany 187
Railway Route Search 5


###### We analyse the apps for Navigation, Social Networking and Reference Genre, which have the highest avg ratings.

After observing the data, we find that though the navigation genre is highly rated, the Google maps and Waze app dominate the market and have very high number of ratings. Making an app in this domain won't be feasible/economical for the company
    

Next we analyse the apps for 'Reference' genre. here the dictionaries and Bible take up most of the ratings. When we observe Social networking apps, the gaints like facebook, instagram take up most of the market but there also a considerable of other social networking apps which people use

###### After observing the trends and interests of Users in the IOS app market, my personal Recommendation would be to create an app which would be a combination of Reference and social networking Genre. An app which provides a platform where users could write books, Share their favourites list and has another discussion panel dedicated for just Dicussing books. users could also upload e-book copies if they have any. Users could add things to their favourites list from a favourite list of another user. A lot of new things could be added to the same app with continuous revisions and user feedback. Overall it's likely that the app would sell and generate Profits and at the same time keep users happy

# Most Popular Apps by genre on Google Play Store. 

#### Below we recommend a profitable app profile for Play Store

In google play data set we have the data about number of installs. Even though the numbers are not precise and open ended like 500+, we can make a calculated prediction of which app genre attracts max users. For that we need to remove , and + characters from the install figures to get it in numeric format. The followinf code does that.

In [765]:
google_freq_table = freq_table(free_android_apps,1)

avg_installs_list=[]
for category in google_freq_table:
    total=0
    len_category=0
    for each in free_android_apps:
        category_app=each[1]
        if category == category_app:
            n_installs=each[5]
            n_installs=n_installs.replace('+','')
            n_installs=n_installs.replace(',','')
            n_install_num=float(n_installs)
            total+=n_install_num
            len_category+=1
            
    avg_n_installs=total/len_category
    avg_installs_list.append([avg_n_installs,category])
    print(category ,':', avg_n_installs ,'\n')
    
    


BOOKS_AND_REFERENCE : 8767811.894736841 

WEATHER : 5074486.197183099 

PHOTOGRAPHY : 17840110.40229885 

NEWS_AND_MAGAZINES : 9549178.467741935 

COMMUNICATION : 38456119.167247385 

LIBRARIES_AND_DEMO : 638503.734939759 

HEALTH_AND_FITNESS : 4188821.9853479853 

SHOPPING : 7036877.311557789 

AUTO_AND_VEHICLES : 647317.8170731707 

TOOLS : 10801391.298666667 

SPORTS : 3638640.1428571427 

DATING : 854028.8303030303 

FINANCE : 1387692.475609756 

BUSINESS : 1712290.1474201474 

PARENTING : 542603.6206896552 

MAPS_AND_NAVIGATION : 4056941.7741935486 

FOOD_AND_DRINK : 1924897.7363636363 

HOUSE_AND_HOME : 1331540.5616438356 

EDUCATION : 1833495.145631068 

PERSONALIZATION : 5201482.6122448975 

VIDEO_PLAYERS : 24727872.452830188 

MEDICAL : 120550.61980830671 

GAME : 15588015.603248259 

TRAVEL_AND_LOCAL : 13984077.710144928 

ENTERTAINMENT : 11640705.88235294 

COMICS : 817657.2727272727 

LIFESTYLE : 1437816.2687861272 

FAMILY : 3695641.8198090694 

ART_AND_DESIGN : 1986335.08

### Below we print the list of average number of installs for each genre in descending order

In [766]:

print(sorted(avg_installs_list,reverse=True))


[[38456119.167247385, 'COMMUNICATION'], [24727872.452830188, 'VIDEO_PLAYERS'], [23253652.127118643, 'SOCIAL'], [17840110.40229885, 'PHOTOGRAPHY'], [16787331.344927534, 'PRODUCTIVITY'], [15588015.603248259, 'GAME'], [13984077.710144928, 'TRAVEL_AND_LOCAL'], [11640705.88235294, 'ENTERTAINMENT'], [10801391.298666667, 'TOOLS'], [9549178.467741935, 'NEWS_AND_MAGAZINES'], [8767811.894736841, 'BOOKS_AND_REFERENCE'], [7036877.311557789, 'SHOPPING'], [5201482.6122448975, 'PERSONALIZATION'], [5074486.197183099, 'WEATHER'], [4188821.9853479853, 'HEALTH_AND_FITNESS'], [4056941.7741935486, 'MAPS_AND_NAVIGATION'], [3695641.8198090694, 'FAMILY'], [3638640.1428571427, 'SPORTS'], [1986335.0877192982, 'ART_AND_DESIGN'], [1924897.7363636363, 'FOOD_AND_DRINK'], [1833495.145631068, 'EDUCATION'], [1712290.1474201474, 'BUSINESS'], [1437816.2687861272, 'LIFESTYLE'], [1387692.475609756, 'FINANCE'], [1331540.5616438356, 'HOUSE_AND_HOME'], [854028.8303030303, 'DATING'], [817657.2727272727, 'COMICS'], [647317.817

###### From the above printed listed we observe Genres like Communication, Video players, Social , Productivity and Games are top 5 genres for Maximum install figures.


In [767]:
for each in free_android_apps:
    if(each[1]=='COMMUNICATION'):
        print(each[0],':',each[5])
    

WhatsApp Messenger : 1,000,000,000+
Messenger for SMS : 10,000,000+
My Tele2 : 5,000,000+
imo beta free calls and text : 100,000,000+
Contacts : 50,000,000+
Call Free – Free Call : 5,000,000+
Web Browser & Explorer : 5,000,000+
Browser 4G : 10,000,000+
MegaFon Dashboard : 10,000,000+
ZenUI Dialer & Contacts : 10,000,000+
Cricket Visual Voicemail : 10,000,000+
TracFone My Account : 1,000,000+
Xperia Link™ : 10,000,000+
TouchPal Keyboard - Fun Emoji & Android Keyboard : 10,000,000+
Skype Lite - Free Video Call & Chat : 5,000,000+
My magenta : 1,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Seznam.cz : 1,000,000+
Antillean Gold Telegram (original version) : 100,000+
AT&T Visual Voicemail : 10,000,000+
GMX Mail : 10,000,000+
Omlet Chat : 10,000,000+
My Vodacom SA : 5,000,000+
Microsoft Edge : 5,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Calls & Text by Mo+ : 5,000,000+
free 

In [768]:
for each in free_android_apps:
    if(each[1]=='VIDEO_PLAYERS'):
        print(each[0],':',each[5])

YouTube : 1,000,000,000+
All Video Downloader 2018 : 1,000,000+
Video Downloader : 10,000,000+
HD Video Player : 1,000,000+
Iqiyi (for tablet) : 1,000,000+
Video Player All Format : 10,000,000+
Motorola Gallery : 100,000,000+
Free TV series : 100,000+
Video Player All Format for Android : 500,000+
VLC for Android : 100,000,000+
Code : 10,000,000+
Vote for : 50,000,000+
XX HD Video downloader-Free Video Downloader : 1,000,000+
OBJECTIVE : 1,000,000+
Music - Mp3 Player : 10,000,000+
HD Movie Video Player : 1,000,000+
YouCut - Video Editor & Video Maker, No Watermark : 5,000,000+
Video Editor,Crop Video,Movie Video,Music,Effects : 1,000,000+
YouTube Studio : 10,000,000+
video player for android : 10,000,000+
Vigo Video : 50,000,000+
Google Play Movies & TV : 1,000,000,000+
HTC Service － DLNA : 10,000,000+
VPlayer : 1,000,000+
MiniMovie - Free Video and Slideshow Editor : 50,000,000+
Samsung Video Library : 50,000,000+
OnePlus Gallery : 1,000,000+
LIKE – Magic Video Maker & Community : 50,

In both the categories we observe that the market is dominated by giants and the games category seems to be saturated with a variety of options.

In [770]:
for each in free_android_apps:
    if(each[1]=='BOOKS_AND_REFERENCE'):
        print(each[0],':',each[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

The books and reference market which showed potential in appstore also shows potential in google play store. Our previouss idea for app (a fusion of Books and social netwroking genre) is still applicable for Google Play store and can provide Profits
